# Chapter 10: Introduction to Artificial Neural Networks with Keras

### 10. Train a deep MLP on the MNIST dataset (you can load it using keras.datasets.mnist.load_data(). See if you can get over 98% precision. Try searching for the optimal learning rate by using the approach presented in this chapter (i.e., by growing the learning rate exponentially, plotting the error, and finding the point where the error shoots up). Try adding all the bells and whistles—save checkpoints, use early stopping, and plot learning curves using TensorBoard.

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import (
    layers,
    callbacks,
    datasets,
)

ImportError: dlopen(/Users/mmenendezg/Developer/Books/.venv/lib/python3.10/site-packages/google/protobuf/pyext/_message.cpython-310-darwin.so, 0x0002): symbol not found in flat namespace (__ZNK6google8protobuf10TextFormat21FastFieldValuePrinter19PrintMessageContentERKNS0_7MessageEiibPNS1_17BaseTextGeneratorE)

Let's first download the dataset and split it into training, validation and test sets. The train set returned by `tf.keras.datasets.mnist.load_data()` has 60k images. We will use 50k for training and 10k for validation. At the same time we will normalize the values of the pixels to make sure all the possible values lay between 0 and 1.

In [ ]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

